# Analyze accounts

In this step we search for insights about the accounts.

In [1]:
import collections
import json
import os

import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters


register_matplotlib_converters()

%matplotlib inline

plt.style.use("seaborn")

## Load the data

In [2]:
data_directory = os.path.join("..", "..", "..", "data")

In [3]:
campaigns_file_path = os.path.join(data_directory, "suppression_campaigns.json")
if not os.path.exists(campaigns_file_path):
    with tarfile.open(os.path.join(data_directory, "suppression_campaigns.tar.xz"), "r", encoding="utf-8") as compressed_file:
        compressed_file.extract("suppression_campaigns.json", data_directory)

In [4]:
with open(campaigns_file_path, "r", encoding="utf-8") as campaigns_file:
    campaigns = []
    line = campaigns_file.readline().strip()
    while line != "":
        campaigns.append(json.loads(line))
        line = campaigns_file.readline().strip()

In [5]:
with open(os.path.join(data_directory, "suppression_bot_hash_to_cluster_id.json"), "r") as json_file:
    bot_hash_to_cluster_id = json.load(json_file)

In [6]:
with open(os.path.join(data_directory, "suppression_attacker_hash_to_cluster_id.json"), "r") as json_file:
    attacker_hash_to_cluster_id = json.load(json_file)

## Collect into data frames

In [7]:
suppressed_contract_names = collections.defaultdict(lambda: set())

# sets
attackers = set()
suppressed_contracts = set()
bots = set()
# campaign counters
attacker_campaign_count = collections.Counter()
suppressed_contract_campaign_count = collections.Counter()
bot_campaign_count = collections.Counter()
# round counters
attacker_round_count = collections.Counter()
suppressed_contract_round_count = collections.Counter()
bot_round_count = collections.Counter()
# transaction counters
attacker_transaction_count = collections.Counter()
suppressed_contract_transaction_count = collections.Counter()
bot_transaction_count = collections.Counter()
# relationships
attacker_bots = collections.defaultdict(lambda: set())
attacker_suppressed_contracts = collections.defaultdict(lambda: set())
bot_attackers = collections.defaultdict(lambda: set())
bot_suppressed_contracts = collections.defaultdict(lambda: set())
suppressed_contract_attackers = collections.defaultdict(lambda: set())
suppressed_contract_bots = collections.defaultdict(lambda: set())
bot_first_blocks = dict()
# profits
attacker_profits = collections.defaultdict(lambda: 0.0)
bot_profits = collections.defaultdict(lambda: 0.0)

for campaign in campaigns:
    # shortcuts
    suppressed_contract = campaign["suppressed_contract_address"]
    bot = campaign["bot_address"]
    rounds = campaign["nr_of_rounds"]
    transactions = campaign["nr_of_transactions"]
    profit = campaign["profit_usd"]
    
    suppressed_contract_names[suppressed_contract] = campaign["suppressed_contract_name"]
    
    # they are never the same entities
    assert suppressed_contract != bot
    
    # update the sets
    suppressed_contracts.add(suppressed_contract)
    bots.add(bot)
    
    # update campaign counters
    suppressed_contract_campaign_count[suppressed_contract] += 1
    bot_campaign_count[bot] += 1
    
    # update round counters
    suppressed_contract_round_count[suppressed_contract] += rounds
    bot_round_count[bot] += rounds
    
    # update transaction counters
    suppressed_contract_transaction_count[suppressed_contract] += transactions
    bot_transaction_count[bot] += transactions
    
    # update the relationships
    suppressed_contract_bots[suppressed_contract].add(bot)
    bot_suppressed_contracts[bot].add(suppressed_contract)
    
    if bot not in bot_first_blocks:
        bot_first_blocks[bot] = campaign["first_block"]
    
    # update the profit
    bot_profits[bot] += profit
    
    # update all for attackers
    manual_transaction_count = 0
    campaign_attackers = set()
    for round_ in campaign["rounds"]:
        round_attackers = set()
        # for transaction attackers
        for transaction in round_["transactions"]:
            assert transaction["to"] == bot
            manual_transaction_count += 1
            
            attacker = transaction["from"]
            attackers.add(attacker)
            campaign_attackers.add(attacker)
            round_attackers.add(attacker)
            
            
            attacker_transaction_count[attacker] += 1
        
        # for round attackers
        for attacker in round_attackers:
            attacker_round_count[attacker] += 1
        
    assert manual_transaction_count == transactions
    
    # for campaign attackers
    for attacker in campaign_attackers:
        attacker_campaign_count[attacker] += 1
        attacker_profits[attacker] += profit
        
        attacker_bots[attacker].add(bot)
        bot_attackers[bot].add(attacker)
        
        attacker_suppressed_contracts[attacker].add(suppressed_contract)
        suppressed_contract_attackers[suppressed_contract].add(attacker)
    
# some numbers
print("number of campaigns", len(campaigns))
print("number of unique attackers involved", len(attackers))
print("number of unique suppressed contracts involved", len(suppressed_contracts))
print("number of unique bots involved", len(bots))

number of campaigns 50
number of unique attackers involved 98
number of unique suppressed contracts involved 15
number of unique bots involved 30


In [8]:
df_attackers = pd.DataFrame({
    "Attacker Hash": pd.Series([], dtype="object"),
    "Campaigns": pd.Series([], dtype="int"),
    "Rounds": pd.Series([], dtype="int"),
    "Transactions": pd.Series([], dtype="int"),
    "Bots": pd.Series([], dtype="int"),
    "Suppressed Contracts": pd.Series([], dtype="int"),
    "Profit": pd.Series([], dtype="float"),
    "Bot Cluster ID": pd.Series([], dtype="int"),
})

for attacker in attackers:
    df_attackers = df_attackers.append({
        "Attacker Hash": attacker,
        "Campaigns": attacker_campaign_count[attacker],
        "Rounds": attacker_round_count[attacker],
        "Transactions": attacker_transaction_count[attacker],
        "Bots": len(attacker_bots[attacker]),
        "Suppressed Contracts": len(attacker_suppressed_contracts[attacker]),
        "Profit": attacker_profits[attacker],
        "Bot Cluster ID": attacker_hash_to_cluster_id.get(attacker),  # will give None if bot clusters are not updated
    }, ignore_index=True)
    
df_attackers.to_csv(os.path.join(data_directory, "suppression_attackers.csv"), index=False)
    
df_attackers.sort_values(by="Profit", ascending=False)

,Attacker Hash,Campaigns,Rounds,Transactions,Bots,Suppressed Contracts,Profit,Bot Cluster ID
34,0x18a0451Ea56Fd4FF58f59837e9EC30f346ffDCa5,1,31,48,1,1,791211.860156,5
74,0x28a421602386603720eF6AB64FE555bb3d1c954e,3,90,131,2,1,777548.671787,5
76,0x45Cf2aE05c1409cecCF4eBa5Fd82D45Ee4CD1CEf,3,81,113,2,1,777548.671787,5
1,0xc80476579Cc83961fdFAa896670F2c4f724490a3,3,93,132,2,1,777548.671787,5
72,0xd71f6B174f7c901C850cf0655950Ce2eCE89904a,3,84,121,2,1,777548.671787,5
...,...,...,...,...,...,...,...,...
11,0x8eab2647099073C4e321b63470CdE02942c8A793,2,13,58,2,2,-11890.915936,2
93,0x83710f74263d96E21A522966e2CEEC9Deba23ccf,2,13,58,2,2,-11890.915936,2
4,0xb7F71F8f3510026928B0631bcb72683129C6d4Fd,2,13,60,2,2,-11890.915936,2
70,0x3e5722cD6d309e5604cea59951c7A733fccc0866,2,13,58,2,2,-11890.915936,2


In [9]:
df_bots = pd.DataFrame({
    "Bot Hash": pd.Series([], dtype="object"),
    "Campaigns": pd.Series([], dtype="int"),
    "Rounds": pd.Series([], dtype="int"),
    "Transactions": pd.Series([], dtype="int"),
    "Attackers": pd.Series([], dtype="int"),
    "Suppressed Contracts": pd.Series([], dtype="int"),
    "Profit": pd.Series([], dtype="float"),
    "Bot Cluster ID": pd.Series([], dtype="int"),
    "First Block": pd.Series([], dtype="int"),
})

for bot in bots:
    df_bots = df_bots.append({
        "Bot Hash": bot,
        "Campaigns": bot_campaign_count[bot],
        "Rounds": bot_round_count[bot],
        "Transactions": bot_transaction_count[bot],
        "Attackers": len(bot_attackers[bot]),
        "Suppressed Contracts": len(bot_suppressed_contracts[bot]),
        "Profit": bot_profits[bot],
        "Bot Cluster ID": bot_hash_to_cluster_id.get(bot),  # will give None if bot clusters are not updated
        "First Block": bot_first_blocks[bot],
    }, ignore_index=True)
    
df_bots.to_csv(os.path.join(data_directory, "suppression_bots.csv"), index=False)

df_bots.sort_values(by="Profit", ascending=False)

,Bot Hash,Campaigns,Rounds,Transactions,Attackers,Suppressed Contracts,Profit,Bot Cluster ID,First Block
7,0x705203fc06027379681AEf47c08fe679bc4A58e1,2,123,4398,41,1,780470.740425,5,6385588
5,0xD6e8a9a1873fa97a75e0b8bc954aa5C698820273,3,3,46,6,1,127585.600257,2,6232122
26,0xdcC655B2665A675B90ED2527354C18596276B0de,4,9,190,20,1,113036.025893,3,6501340
25,0xd037763925C23f5Ba592A8b2F4910D051a57A9e3,11,24,321,15,3,51392.604792,4,6198235
19,0x184602ab29aD4618B2668f748f7F359fa31e2ea6,1,4,82,19,1,12484.588601,3,6678018
23,0xD83CF6613FF1404c21bC636FB1A3F754ccDf9717,1,1,49,18,1,3647.683776,3,7640248
3,0xcB4446a431e0b28e85d5DbB2FF1B90EB222CbB16,1,3,44,18,1,3204.791793,3,7674373
29,0x5d3F186Bf6839cAC0a67a71695860E65AF644985,2,30,376,7,1,2280.930631,2,6208953
28,0x150D2464e0D46D1644C715E7B71Aa989De147b2a,1,1,15,13,1,516.800471,3,6897508
4,0xe3b9c23f6FA9D1DFCDc35a79A48a45d23EB241e0,1,1,7,7,1,-4.667298,3,7168047


In [10]:
pd.concat([
    df_bots[df_bots["Bot Cluster ID"] == 0][["Bot Cluster ID", "Campaigns", "Rounds", "Profit"]],
    df_bots[df_bots["Bot Cluster ID"] > 0].groupby("Bot Cluster ID").agg({
        "Campaigns": "sum",
        "Rounds": "sum",
        "Profit": "sum",
    }).reset_index()
], axis=0).sort_values(by="Profit", ascending=False)

,Bot Cluster ID,Campaigns,Rounds,Profit
4,5,3,136,777548.671787
2,3,18,52,124243.352133
1,2,12,55,115241.447079
3,4,16,87,19274.305876
0,1,1,1,-45.996021


In [11]:
df_suppresed_contracts = pd.DataFrame({
    "Suppressed Contract Address": pd.Series([], dtype="object"),
    "Contract Name": pd.Series([], dtype="object"),
    "Attacks": pd.Series([], dtype="int"),
    "Rounds": pd.Series([], dtype="int"),
    "Transactions": pd.Series([], dtype="int"),
    "Attackers": pd.Series([], dtype="int"),
    "Bots": pd.Series([], dtype="int"),
    "Attacker Clusters": pd.Series([], dtype="int"),
})

for suppressed_contract in suppressed_contracts:
    df_suppresed_contracts = df_suppresed_contracts.append({
        "Suppressed Contract Address": suppressed_contract,
        "Contract Name": suppressed_contract_names[suppressed_contract],
        "Attacks": suppressed_contract_campaign_count[suppressed_contract],
        "Rounds": suppressed_contract_round_count[suppressed_contract],
        "Transactions": suppressed_contract_transaction_count[suppressed_contract],
        "Attackers": len(suppressed_contract_attackers[suppressed_contract]),
        "Bots": len(suppressed_contract_bots[suppressed_contract]),
        "Attacker Clusters": len({bot_hash_to_cluster_id.get(bot)  # will give None if bot clusters are not updated
                             for bot in suppressed_contract_bots[suppressed_contract]}),
    }, ignore_index=True)


def rename_contract_name(row):
    if row["Suppressed Contract Address"] == "0x5D0d76787D9d564061dD23f8209F804a3b8AD2F2":
        row = pd.Series(row)
        row["Contract Name"] = "Peach Will"
    return row
    
df_suppresed_contracts = df_suppresed_contracts.apply(rename_contract_name, axis=1)

df_suppresed_contracts = df_suppresed_contracts.sort_values(by=[
    "Attacks",
    "Transactions",
], ascending=False)

df_suppresed_contracts

,Suppressed Contract Address,Contract Name,Attacks,Rounds,Transactions,Attackers,Bots,Attacker Clusters
0,0xDd9fd6b6F8f7ea932997992bbE67EabB3e316f3C,Last Winner,16,20,304,27,5,2
4,0xA62142888ABa8370742bE823c1782D17A0389Da1,FoMo3Dlong,12,188,5875,81,8,4
8,0x5D0d76787D9d564061dD23f8209F804a3b8AD2F2,Peach Will,6,52,1105,26,5,2
5,0x2c58B11405a6a8154FD3bbC4CcAa43924f2BE769,ERD,3,3,207,20,2,1
12,0x42CeaD70158235a6ca4868F3CFAF600c7A7b0ebB,ETH CAT,2,23,929,20,2,1
13,0xB7C2e4047Fb76508D4137BE787DaF28B013F00E6,Escape plan,2,3,67,20,2,1
9,0x29488e24cFdAA52a0b837217926C0c0853Db7962,SuperCard,1,25,319,17,1,1
7,0xB4a448387403554616eB5B50aa4C48f75243a015,Mobius2Dv2,1,4,82,19,1,1
11,0x3e22bB2279d6Bea3Cfe57f3Ed608fC3B1DeaDADf,Star3Dlong,1,3,66,6,1,1
1,0xD15E559f6BD5C785Db35E550F9FbC80045b0a049,FDC,1,3,44,18,1,1


## Save latex tables for the publication

In [12]:
with pd.option_context("display.max_colwidth", 10000):
    latex = df_suppresed_contracts.to_latex(index=False, escape=False, formatters={
        "Suppressed Contract Address": lambda value: "\\href{{https://etherscan.io/address/{}}}{{{}}}".format(value, value),
        "Contract Name": lambda value: value,
        "Attacks": str,
        "Rounds": str,
        "Transactions": str,
        "Attacker Accounts": str,
        "Bots Contracts": str,
        "Attacker Clusters": str,
    })

    with open(os.path.join(data_directory, "suppression_victim_stats.tex"), "w") as latex_file:
        latex_file.write(latex)